<h1>Scraping fidelity.com</h1>
In this assignment, you will scrape data from fidelity.com. The goal of the exercise is to get the latest sector performance data from the US markets, and to get the total market capitalization for each sector. 

The end result is to write a function: <i>get_us_sector_performance()</i> that will return a list of tuples. Each tuple should correspond to a sector and should contain the following data:
<li>the sector name
<li>the amount the sector has moved
<li>the market capitalization of the sector
<li>the market weight of the sector
<li>a link to the fidelity page for that sector

<p>
The data should be sorted by decreasing order of market weight. I.e., the sector with the highest weight should be in the first tuple, etc.

<h3>Process</h3>
<li>Get a list of sectors and the links to the sector detail pages from the url (see function)
<li>Loop through the list and call the function <i>get_sector_change_and_market_cap(sector_page_link)</i> for each sector
<li>Accumulate the name, the change, the capitalization, the weight and the link for each sector in output_list (see function)
<li>Sort the list by market weight

<b>Notes:</b>
<li>Note that the market weight is a string with a % sign at the back. You will need to get rid of the % and convert the string into a float before you can sort it
<li>Your starting data is the url listed below. You need to extract all data, including links to the sector pages, from the page at this url
<li>To sort a list of tuples by an arbitrary element, use the example at the bottom of this notebook

In [9]:
def get_us_sector_performance():
    import requests
    from bs4 import BeautifulSoup
    output_list = list()
    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    
    # get each sector
    sector_list = soup.find_all('a', {'class':'heading1'})

    # looks through each sector and gets its hyperlink in order to use get_sector_change_and_market_cap
    for sector in sector_list:
        href = sector.get('href')
        new_url = 'https://eresearch.fidelity.com/' + href
        
        # saves tuple output from get_sector_change_and_market_cap
        sector_change_and_market_cap = get_sector_change_and_market_cap(new_url)
        
        # appends each tuple to the output list
        output_list.append((sector.get_text().strip(), sector_change_and_market_cap[0], sector_change_and_market_cap[1], 
                            sector_change_and_market_cap[2], new_url))

    # sorts the output list by market weight in descending order
    output_list.sort(reverse=True, key = lambda x: x[3])
    
    return output_list
    

In [10]:
def get_sector_change_and_market_cap(sector_page_link):
    import requests
    from bs4 import BeautifulSoup
    response = requests.get(sector_page_link)
    soup = BeautifulSoup(response.content, 'lxml')
    
    # checks if the sector_change is positive or negative and gets either the positive or negative number
    if soup.find('span', {'class':'negative'}) == None:
        sector_change = soup.find('span', {'class':'positive'})
    else:
        sector_change = soup.find('span', {'class':'negative'})
        
    # strips the '%' off the value
    sector_change = sector_change.get_text().strip()[:-1]
    # converts the string into a float to match the desired output
    sector_change = float(sector_change)
    
    # finds everything with a 'td' tag and then uses the 2nd string which has the market cap and partitions the string into
    # two lines so that the date can be easily excluded and then takes just the market cap value
    sector_market_cap = soup.find_all('td')[1].get_text().strip().partition('\n')[0]
    
    # finds the 'td', {'class':'left-spcng'} tag corresponding to market weight and partitions the string into two lines
    # so that the date can be easily excluded and then takes just the market weight and removes the '%' so it can be made into
    # a float to match the output and so the tuples can be sorted
    sector_market_weight = soup.find('td', {'class':'left-spcng'}).get_text().strip().partition('\n')[0][:-1]
    sector_market_weight = float(sector_market_weight)
    
    return sector_change,sector_market_cap,sector_market_weight

In [11]:
#Test get_sector_change_and_market_cap()
link = "https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25"
get_sector_change_and_market_cap(link)
#Should return something like (-1.0, '$5.49T', 10.24)
#Note that the conversion to int of the change and the weight

(0.83, '$5.23T', 10.08)

In [12]:
#Test get_us_sector_performance()
get_us_sector_performance()

[('Information Technology',
  1.28,
  '$7.90T',
  21.82,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45'),
 ('Health Care',
  1.37,
  '$5.39T',
  13.6,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Financials',
  0.14,
  '$7.19T',
  13.02,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Communication Services',
  0.43,
  '$4.54T',
  10.38,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Consumer Discretionary',
  0.83,
  '$5.23T',
  10.08,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Industrials',
  0.37,
  '$4.12T',
  9.37,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20')

<h2>Example of the return value</h2>
<pre>
[('Materials',
  -1.19,
  '$1.88T',
  2.67,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15'),
 ('Real Estate',
  0.45,
  '$1.37T',
  3.25,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),
 ('Utilities',
  -0.05,
  '$1.45T',
  3.42,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),
 ('Energy',
  2.8,
  '$3.13T',
  4.45,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),
 ('Consumer Staples',
  -0.72,
  '$3.94T',
  7.54,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'),
 ('Industrials',
  -0.07,
  '$4.22T',
  9.23,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 ('Consumer Discretionary',
  -0.99,
  '$5.49T',
  10.24,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Communication Services',
  -0.61,
  '$4.70T',
  10.48,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Financials',
  -0.45,
  '$7.18T',
  12.79,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Health Care',
  -0.1,
  '$5.49T',
  13.74,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Information Technology',
  -0.15,
  '$8.13T',
  22.19,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45')]
  </pre>

<h3>Sorting</h3>
<li>the <span style="color:red">sort</span> function sorts a list "in-place". I.e., the list itself changes so that the contents are in sorted order</li>
<li>the <span style="color:red">sorting</span> function returns a new sorted list</li>
<li>both functions take arguments that determine the key (<span style="color:red">key=</span>) and the order. Ascending is the default order, to flip it use <span style="color:red">reverse=True</span></li>
<li>sort and sorted will only work if the data is sortable. For example, sorted([1,9,2,8,11,'a']) will not work because an integer and a string cannot be ordered. In Python, the order operator is the < (less than) sign</li>

In [125]:
x = [1,9,2,8,11]
x.sort()
print(x) #x is now a sorted list
x.sort(reverse=True)
print(x) #x is not sorted in descending order

y = [1,9,2,8,11]
sorted(y)
print(y) #y is unchanged because sorted(y) returned a new list

z = sorted(y)
print(z) #z contains the sorted contents of y

[1, 2, 8, 9, 11]
[11, 9, 8, 2, 1]
[1, 9, 2, 8, 11]
[1, 2, 8, 9, 11]


<h3>comparing tuples</h3

In [126]:
(1,2,3,5) < (1,2,4,1) 
#Python does an element wise comparison, somewhat like when strings are compared
#When sorting tuples (or lists), that element wise comparison is used to determine ordering

True

In [127]:
x = [('c',17.4,'f'),('e',1.74,'bb'),('d',29.2,'z'),('a',23.2,'b'),('d',29.2,'a')]
x.sort() #Sorts using tuple comparison, going left to right. Note the order of the 'd' tuples
x

[('a', 23.2, 'b'),
 ('c', 17.4, 'f'),
 ('d', 29.2, 'a'),
 ('d', 29.2, 'z'),
 ('e', 1.74, 'bb')]

<li>When a key is specified, sort (and sorted) will only use that key
<li>In the example below, note that the two 'd' tuples are not re-ordered (because 'z' and 'a' are not compared)
<li>sort and sorted are stable sort functions. If two elements are "equal" (per the sorting rule), they will be returned in the same order as they were in the original array

In [128]:
x = [('c',17.4,'f'),('e',1.74,'bb'),('d',29.2,'z'),('a',23.2,'b'),('d',29.2,'a')]
x.sort(key=lambda k: k[1]) 
x

[('e', 1.74, 'bb'),
 ('c', 17.4, 'f'),
 ('a', 23.2, 'b'),
 ('d', 29.2, 'z'),
 ('d', 29.2, 'a')]

In [129]:
#If you don't "get" lambda functions!
#itemgetter returns the item at the specified index in a collection
#the key= below tells sort to use the element at location 1 when comparing elements in x
from operator import itemgetter
x = [('c',17.4,'f'),('e',1.74,'bb'),('d',29.2,'z'),('a',23.2,'b'),('d',29.2,'a')]
x.sort(key=itemgetter(1)) 
x

[('e', 1.74, 'bb'),
 ('c', 17.4, 'f'),
 ('a', 23.2, 'b'),
 ('d', 29.2, 'z'),
 ('d', 29.2, 'a')]